## 深度优先搜索（DFS）

### 多叉树深度遍历（点）-- visited 数组

In [ ]:
# 多叉树节点
class Node:
    def __init__(self, val=0, children=None):
        self.val = val
        self.children = children if children is not None else []

# 多叉树遍历框架
def traverse(root:Node):
    if root is None:
        return

    # 前序位置
    print(f"visit: {root.val}")

    for child in root.children:
        traverse(child)
    # 后序位置

### 图的深度遍历（点）-- visited 数组

由于 visited 数组的剪枝作用，这个遍历函数会遍历一次图中的所有节点，并尝试遍历一次所有边，所以算法的时间复杂度是 O(E+V)，其中 E 是边的总数，V 是节点的总数。

- 每条边触发一次遍历操作（E 条边）
- 访问节点也是一次操作（V 个节点）


In [ ]:
from typing import List

# 1.遍历所有节点
class Vertex:
    def __init__(self, id=0, neighbors=None):
        self.id = id
        self.neighbors = neighbors if neighbors is not None else []
    
# 遍历
# visited 记录已经走过的
def traverse_graph(s: Vertex, visited: List[bool]):
    if s is None:
        return
    
    # 前序位置
    visited[s.id] = True
    print(f"visit {s.id}")

    for neighbor in s.neighbors:
        traverse_graph(neighbor, visited)
    # 后序位置


#========================================================
# 遍历图的所有节点
def traverse(graph, s, visited):
    # base case
    if s < 0 or s >= len(graph):
        return
    if visited[s]:
        # 防止死循环
        return
    # 前序位置
    visited[s] = True
    print("visit", s)
    for e in graph.neighbors(s):
        traverse(graph, e.to, visited)
    # 后序位置

### 多叉树深度遍历（边）-- 二维 visited 数组

In [ ]:
# 遍历多叉树的树枝
def traverse_branch(root: Node):
    if root is None:
        return
    
    for child in root.children:
        print(f"visit branch: {root.val} -> {child.val}")
        traverse_branch(child)

### 图的深度遍历（边）-- 二维 visited 数组

由于一条边由两个节点构成，所以我们需要把前序位置的相关代码放到 for 循环内部。

In [ ]:
# 2.遍历所有边
# visited[u][v]表示边 u->v 已经被遍历
# 由于一条边由两个节点构成，所以我们需要把前序位置的相关代码放到 for 循环内部。
def traverse_edges(s: Vertex, visited):
    if s is None:
        return
    
    for neighbor in s.neighbors:
        if visited[s.id][neighbor.id]:
            continue

        visited[s.id][neighbor.id] = True
        print(f"visit edge: {s.id} -> {neighbor.id}")
        traverse_edges(neighbor, visited)


def traverse_edges(graph, s, visited):
    # base case
    if s < 0 or s >= len(graph):
        return
    
    for e in graph.neighbors(s):
        # 如果边已经被遍历过，则跳过
        if visited[s][e.to]:
            continue
        # 标记并访问边
        visited[s][e.to] = True
        print(f"visit edge: {s} -> {e.to}")
        traverse_edges(graph, e.to, visited)

显然，使用二维 visited 数组并不是一个很高效的实现方式，因为需要创建二维 visited 数组，这个算法的时间复杂度是 O(E+V^2)，空间复杂度是 O(V^2)，其中 E 是边的数量，V 是节点的数量。

### 遍历所有路径（onPath 数组）

对于树结构，遍历所有「路径」和遍历所有「节点」是没什么区别的。而对于图结构，遍历所有「路径」和遍历所有「节点」稍有不同。

因为对于树结构来说，只能由父节点指向子节点，所以从根节点 root 出发，到任意一个节点 targetNode 的路径都是唯一的。换句话说，我遍历一遍树结构的所有节点之后，必然可以找到 root 到 targetNode 的唯一路径：



In [ ]:
# 多叉树的遍历框架，寻找从根节点到目标节点的路径
path = []

def traverse(root, targetNode):
    # base case
    if root is None:
        return
    if root.val == targetNode.val:
        # 找到目标节点
        print(f"find path: {'->'.join(path)}->{targetNode}")
        return
    # 前序位置
    path.append(root)
    for child in root.children:
        traverse(child, targetNode)
    # 后序位置
    path.pop()

而对于图结构来说，由起点 src 到目标节点 dest 的路径可能不止一条。我们需要一个 onPath 数组，在进入节点时（前序位置）标记为正在访问，退出节点时（后序位置）撤销标记，这样才能遍历图中的所有路径，从而找到 src 到 dest 的所有路径：

In [ ]:
# 下面的算法代码可以遍历图的所有路径，寻找从 src 到 dest 的所有路径

# onPath 和 path 记录当前递归路径上的节点
on_path = [False] * len(graph)
path = []

def traverse(graph, src, dest):
    # base case
    if src < 0 or src >= len(graph):
        return
    if on_path[src]:
        # 防止死循环（成环）
        return
    if src == dest:
        # 找到目标节点
        print(f"find path: {'->'.join(map(str, path))}->{dest}")
        return

    # 前序位置
    on_path[src] = True
    path.append(src)
    for e in graph.neighbors(src):
        traverse(graph, e.to, dest)
    # 后序位置
    path.pop()
    on_path[src] = False

### 完全不用 visited 和 onPath

In [ ]:
class Solution:
    def __init__(self):
        self.res = []  # 存很多条下面的 path，使用 copy()
        self.path =[]

    def allPathSourcetarget(self, graph: List[List[int]]) -> List[List[int]]:
        self.traverse(graph, 0)
        return self.res
    
    def traverse(self, graph: List[List[int]], s: int):
        # 添加节点 s 到路径
        self.path.append(s)

        n = len(graph)

        if s == n - 1:
            # 到达终点
            self.res.append(self.path.copy())
            self.path.pop()
            return
        
        # 递归每个相邻节点
        for v in graph[s]:
            self.traverse(graph, v)

        self.path.pop()

## 广度优先算法

### 不记录遍历步数

In [ ]:
from collections import deque

def level_order_traverse(root):
    if root is None:
        return
    
    q = deque()
    q.append(root)

    while q:
        cur = q.popleft()

        print(cur.val)

        for child in cur.children:
            q.append(child)


def bfs(graph, s):
    visited = [False] * len(graph)
    q = deque([s])
    visited[s] = True

    while q:
        cur = q.popleft()
        print(cur)

        for e in graph.neighbors(cur):
            if visited[e.to]:
                continue
            q.append(e.to)
            visited[e.to] = True

### 记录步数

In [ ]:
from collections import deque

def levelOrderTraverse(root):
    if root is None:
        return
    q = deque()
    q.append(root)
    # 记录当前遍历到的层数（根节点视为第 1 层）
    depth = 1

    while q:
        sz = len(q)
        for i in range(sz):
            cur = q.popleft()
            # 访问 cur 节点，同时知道它所在的层数
            print(f"depth = {depth}, val = {cur.val}")

            for child in cur.children:
                q.append(child)
        depth += 1



from collections import deque

# 从 s 开始 BFS 遍历图的所有节点，且记录遍历的步数
def bfs(graph, s):
    visited = [False] * len(graph) 
    q = deque([s])
    visited[s] = True
    # 记录从 s 开始走到当前节点的步数
    step = 0
    
    while q:
        sz = len(q)
        for i in range(sz):
            cur = q.popleft()
            print(f"visit {cur} at step {step}")
            for e in graph.neighbors(cur):
                if visited[e.to]: 
                    continue
                q.append(e.to)
                visited[e.to] = True
        step += 1

### 适配权重

In [ ]:
# 多叉树的层序遍历
# 每个节点自行维护 State 类，记录深度等信息
class State:
    def __init__(self, node, depth):
        self.node = node
        self.depth = depth

def levelOrderTraverse(root):
    if root is None:
        return
    q = deque()
    # 记录当前遍历到的层数（根节点视为第 1 层）
    q.append(State(root, 1))

    while q:
        state = q.popleft()
        cur = state.node
        depth = state.depth
        # 访问 cur 节点，同时知道它所在的层数
        print(f"depth = {depth}, val = {cur.val}")

        for child in cur.children:
            q.append(State(child, depth + 1))


# 图结构的 BFS 遍历，从节点 s 开始进行 BFS，且记录遍历步数（从起点 s 到当前节点的边的条数）
# 每个节点自行维护 State 类，记录从 s 走来的遍历步数
class State:
    def __init__(self, node, step):
        # 当前节点 ID
        self.node = node
        # 从起点 s 到当前节点的遍历步数
        self.step = step


def bfs(graph, s):
    visited = [False] * len(graph)
    from collections import deque

    q = deque([State(s, 0)])
    visited[s] = True

    while q:
        state = q.popleft()
        cur = state.node
        step = state.step
        print(f"visit {cur} with step {step}")
        for e in graph.neighbors(cur):
            if visited[e.to]:  
                continue
            q.append(State(e.to, step + 1))
            visited[e.to] = True